# Demand Planning KPI Monitor

In [ ]:
import os, sys, gc, datetime, time

import pandas as pd
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
from pandas.plotting import register_matplotlib_converters
from pandas import ExcelWriter

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib as mpl
%matplotlib inline

import seaborn as sns
plt.style.use('seaborn')
%config InlineBackend.figure_format = 'retina'

register_matplotlib_converters()

from pyspark.sql import SparkSession
from impala.dbapi import connect

In [ ]:
## Get parameters, if not given then fall back to default values

tfmt = '%Y%m%d'
_end = (datetime.datetime.now() - datetime.timedelta(days=1)).strftime(tfmt)

if 'MONITOR_RUN_DATE' in os.environ:
    print('Using external parameters.')
    _end = os.environ.get('MONITOR_RUN_DATE')
else:
    print('Using default parameters.')
    
_start = (datetime.datetime.strptime(_end, '%Y%m%d').date() - datetime.timedelta(days=60)).strftime(tfmt)
date_str = _end

DETENTION_START, DETENTION_END = _start, _end
SERVICE_LEVEL_START, SERVICE_LEVEL_END = _start, _end
STOCK_LEVEL_START, STOCK_LEVEL_END = _start, _end
CONSISTENCY_START, CONSISTENCY_END = _start, _end
OOS_CHECK_DATE = _end

print('Detention:', DETENTION_START, DETENTION_END, sep='\t')
print('Stock level:', STOCK_LEVEL_START, STOCK_LEVEL_END, sep='\t')
print('Consistency:', CONSISTENCY_START, CONSISTENCY_END, sep='\t')
print('Service level:', SERVICE_LEVEL_START, SERVICE_LEVEL_END, sep='\t')
print('OOS_CHECK_DATE:', OOS_CHECK_DATE, sep='\t')

In [ ]:
record_folder = '/data/jupyter/Carrefour-China-Supply-Chain-Forecast/output/monitoring/'

detention_rate_dc_file = f"report_detention_rate_dc_{date_str}.xlsx"

detention_rate_store_file = f"report_detention_rate_store_{date_str}.xlsx"

stock_level_dc_file = f"report_stock_level_dc_{date_str}.xlsx"

stock_level_store_file = f"report_stock_level_store_{date_str}.xlsx"

service_level_file = f"report_service_level_{date_str}.xlsx"

consistency_file = f'report_consistency_items_{date_str}.xlsx'

oos_file = f'report_oos_item_list_{date_str}.xlsx'

In [ ]:
print('Report generation time:', datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), end='\n\n')
T0 = time.time()

---

In [ ]:
os.environ["PYSPARK_SUBMIT_ARGS"] = '--jars /data/jupyter/kudu-spark2_2.11-1.8.0.jar pyspark-shell'
warehouse_location = os.path.abspath('spark-warehouse')

spark = SparkSession \
    .builder \
    .appName("Forecast monitoring process") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("spark.blacklist.enabled", False) \
    .config("spark.driver.memory", '6g') \
    .config("spark.executor.memory", '6g') \
    .config("spark.num.executors", '14') \
    .enableHiveSupport() \
    .getOrCreate()

In [ ]:
kudu_tables = [
    'lfms.daily_dctrxn', 'lfms.daily_dcstock', 'lfms.ord', 'lfms.daily_shipment'
]

for tbl in kudu_tables:
    spark.read.format('org.apache.kudu.spark.kudu') \
    .option('kudu.master', "dtla1apps11:7051,dtla1apps12:7051,dtla1apps13:7051") \
    .option('kudu.table', f'impala::{tbl}') \
    .load() \
    .registerTempTable('{}'.format(tbl.replace('.', '_')))

In [ ]:
def get_query(sql_path, kudu_replace=None, **query_params):
    with open(sql_path, 'r') as f:
        query = f.read()
    if kudu_replace is not None:
        for k, v in kudu_replace.items():
            query = query.replace(k, v)
   
    query = query.format(**query_params)

    return query

In [ ]:
def read_query_and_fetch(sql_path, create_table=False, get_query=False, kudu_replace=None, **query_params):
    with open(sql_path, 'r') as f:
        query = f.read()
    if kudu_replace is not None:
        for k, v in kudu_replace.items():
            query = query.replace(k, v)
    if not create_table:
        ## remove lines with `table`
        q0 = query
        query = '\n'.join([line for line in q0.split('\n')
                           if ('drop table' not in line.lower())
                           and ('create table' not in line.lower())])
    query = query.format(**query_params)
    if get_query:
        return query
    return spark.sql(query).toPandas()

In [ ]:
def run_sql_with_impala(sql):
    with connect(host='dtla1apps14', port=21050, auth_mechanism='PLAIN', user='CHEXT10211', password='datalake2019',
                 database='vartefact') as conn:
        curr = conn.cursor()
        curr.execute(sql)

In [ ]:
dc_stock_sql = get_query(
    'sql/record_dc_stock.sql',
    database_name='vartefact', date_start=DETENTION_START, date_end=DETENTION_END
)

In [ ]:
run_sql_with_impala(dc_stock_sql.replace("\n", " "))

In [ ]:
store_stock_sql = get_query(
    'sql/record_store_stock.sql',
    database_name='vartefact', date_start=DETENTION_START, date_end=DETENTION_END
)

In [ ]:
run_sql_with_impala(store_stock_sql.replace("\n", " "))

---

# 1. Detention rate

### 1.1 Detention rate - DC

In [ ]:
detention_dc = spark.sql(get_query(
    'sql/kpi_detention_rate_dc.sql',
    database='vartefact', run_date=STOCK_LEVEL_END
)).toPandas()

In [ ]:
detention_dc_supplier = spark.sql(get_query(
    'sql/kpi_detention_rate_dc_supplier.sql',
    database='vartefact', run_date=STOCK_LEVEL_END
)).toPandas()

In [ ]:
detention_dc_writer = ExcelWriter(record_folder + detention_rate_dc_file)

In [ ]:
def plot_detention_dc(df, title):
    start_day = pd.to_datetime('20190909', format='%Y%m%d')
    df1 = df.sort_values(by=['rotation', 'date_key']).copy()
    df1['date_dt'] = pd.to_datetime(df1.date_key, format='%Y%m%d')
    fig, ax = plt.subplots(figsize=(12, 3))
    flow_A = df1[df1.rotation == 'A']
    flow_B = df1[df1.rotation == 'B']
    ax.plot(flow_A.date_dt, flow_A.detention_rate, label='Flow A')
    ax.plot(flow_B.date_dt, flow_B.detention_rate, label='Flow B')
    ax.legend()
    ax.set_title(f'DC detention rate for {title}')
    ax.axvline(start_day, ls="--")
    fig.autofmt_xdate()
    
    print(f'Latest detention rate for {title}:')
    display(df[['date_key', 'rotation', 'detention_rate']].tail(9).style.hide_index())
    
    df1.to_excel(detention_dc_writer, sheet_name=title, index=False)

In [ ]:
plot_detention_dc(detention_dc, 'all items')

In [ ]:
plot_detention_dc(detention_dc_supplier[detention_dc_supplier['holding_code'] == '002'], 'Nestle')

In [ ]:
plot_detention_dc(detention_dc_supplier[detention_dc_supplier['holding_code'] == '693'], 'P&G')

In [ ]:
plot_detention_dc(detention_dc_supplier[detention_dc_supplier['holding_code'] == '700'], 'Unilever')

In [ ]:
detention_dc_writer.save()
print(f'Please check file {detention_rate_dc_file} for detail')

---

### 1.2 Detention rate - Store

In [ ]:
detention_all_store = spark.sql(get_query(
    'sql/kpi_detention_rate_all_store.sql',
    database='vartefact', run_date=STOCK_LEVEL_END
)).toPandas()

In [ ]:
detention_store_supplier = spark.sql(get_query(
    'sql/kpi_detention_rate_store_supplier.sql',
    database='vartefact', run_date=STOCK_LEVEL_END
)).toPandas()

In [ ]:
detention_store = spark.sql(get_query(
    'sql/kpi_detention_rate_store.sql',
    database='vartefact', run_date=STOCK_LEVEL_END
)).toPandas()

In [ ]:
detention_store_writer = ExcelWriter(record_folder + detention_rate_store_file)

In [ ]:
def plot_detention_all_store(df, title):
    start_day = pd.to_datetime('20190909', format='%Y%m%d')
    df1 = df.sort_values(by=['rotation', 'date_key']).copy()
    df1['date_dt'] = pd.to_datetime(df1.date_key, format='%Y%m%d')
    
    fig, ax = plt.subplots(figsize=(12, 3))
    flow_A = df1[df1.rotation == 'A']
    flow_B = df1[df1.rotation == 'B']
    flow_X = df1[df1.rotation == 'X']
    
    ax.plot(flow_A.date_dt, flow_A.detention_rate, label='Flow A')
    ax.plot(flow_B.date_dt, flow_B.detention_rate, label='Flow B')
    ax.plot(flow_X.date_dt, flow_X.detention_rate, label='Flow X')
    ax.legend()
    ax.axvline(start_day, ls="--")
    ax.set_title(f'{title} store detention rate by day')
    fig.autofmt_xdate()
    print(f'Latest detention rate:')

    display(df[['date_key', 'rotation', 'detention_rate']].tail(9).style.hide_index())
    
    flow_A.to_excel(detention_store_writer, sheet_name=f"{title} Flow A", index=False)
    flow_B.to_excel(detention_store_writer, sheet_name=f"{title} Flow B", index=False)
    flow_X.to_excel(detention_store_writer, sheet_name=f"{title} Flow X", index=False)

In [ ]:
def plot_detention_store(df):
    df1 = df.sort_values(by=['rotation', 'date_key', 'store_code']).copy()
    df1['date_dt'] = pd.to_datetime(df1.date_key, format='%Y%m%d')

    dr = df1.groupby(['store_code', 'rotation'])['detention_rate'].mean().reset_index()
    a = dr.loc[dr.rotation == 'A', 'detention_rate'].sort_values().values
    b = dr.loc[dr.rotation == 'B', 'detention_rate'].sort_values().values
    x = dr.loc[dr.rotation == 'X', 'detention_rate'].sort_values().values
    
    fig, axes = plt.subplots(figsize=(16, 4), ncols=3)

    axes[0].plot(a, 'o', ms=6)
    axes[1].plot(b, 'o', ms=6)
    axes[2].plot(x, 'o', ms=6)
    
    axes[0].set_title('Store detention rate by store: Flow A')
    axes[1].set_title('Store detention rate by store: Flow B')
    axes[2].set_title('Store detention rate by store: Flow X')
    
    axes[0].set_ylim(min(0.75, a.min()), 1)
    axes[1].set_ylim(min(0.75, b.min()), 1)
    axes[2].set_ylim(min(0.75, x.min()), 1)
    
    fig.tight_layout()
    
    df1[df1.rotation == 'A'].to_excel(detention_store_writer, sheet_name="By store flow A", index=False)
    df1[df1.rotation == 'B'].to_excel(detention_store_writer, sheet_name="By store flow B", index=False)
    df1[df1.rotation == 'X'].to_excel(detention_store_writer, sheet_name="By store flow X", index=False)

In [ ]:
plot_detention_all_store(detention_all_store, "All")

In [ ]:
plot_detention_all_store(detention_store_supplier[detention_store_supplier['con_holding'] == '002'], "Nestle")

In [ ]:
plot_detention_all_store(detention_store_supplier[detention_store_supplier['con_holding'] == '693'], "P&G")

In [ ]:
plot_detention_all_store(detention_store_supplier[detention_store_supplier['con_holding'] == '700'], "Unilever")

In [ ]:
plot_detention_store(detention_store)

In [ ]:
detention_store_writer.save()
print(f'Please check file {detention_rate_store_file} for detail')

---

## 2 Stock level

### 2.1 Stock level - DC

In [ ]:
stock_level_dc = spark.sql(get_query(
    'sql/kpi_stock_level_dc.sql',
    # database_name='vartefact', date_start='20190630', date_end='20190730',
    database='vartefact', run_date=STOCK_LEVEL_END
)).toPandas()

In [ ]:
stock_dc_writer = ExcelWriter(record_folder + stock_level_dc_file)

In [ ]:
def plot_stock_level_dc(df, title):
    start_day = pd.to_datetime('20190909', format='%Y%m%d')
    df1 = df.sort_values(by=['rotation', 'date_key']).copy()
    sl = df1.groupby(['in_dm', 'rotation', 'date_key'])['stock_level'].sum().reset_index()
    sl['date_key'] = pd.to_datetime(sl.date_key, format='%Y%m%d')
    
    sla = df1.groupby(['rotation', 'date_key'])['stock_level'].sum().reset_index()
    sla['date_key'] = pd.to_datetime(sla.date_key, format='%Y%m%d')
    
    fig, axes = plt.subplots(figsize=(12, 4), ncols=2)
    for i, rotation in enumerate(['A', 'B']):
        axes[i].set_title(f'DC stock level {title} Rotation {rotation}')
        axes[i].axvline(start_day, ls="--")
        axes[i].yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
        for dm in sl.in_dm.unique():
            d = sl[(sl.in_dm == dm) & (sl.rotation == rotation)]
            axes[i].plot(d.date_key, d.stock_level, label="DM" if dm else "Non-DM")
            axes[i].legend()
            
        d = sla[(sla.rotation == rotation)]
        axes[i].plot(d.date_key, d.stock_level, label="All")
        axes[i].legend()
        
        
    fig.autofmt_xdate(); fig.tight_layout()
    
    sl_value = df1.groupby(['in_dm', 'rotation', 'date_key'])['stock_value'].sum().reset_index()
    sl_value['date_key'] = pd.to_datetime(sl_value.date_key, format='%Y%m%d')
    
    sla_value = df1.groupby(['rotation', 'date_key'])['stock_value'].sum().reset_index()
    sla_value['date_key'] = pd.to_datetime(sla_value.date_key, format='%Y%m%d')
    
    fig_value, axes_value = plt.subplots(figsize=(12, 4), ncols=2)
    for i, rotation in enumerate(['A', 'B']):
        axes_value[i].set_title(f'DC stock value {title} Rotation {rotation}')
        axes_value[i].yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
        axes_value[i].axvline(start_day, ls="--")
        for dm in sl_value.in_dm.unique():
            d = sl_value[(sl_value.in_dm == dm) & (sl_value.rotation == rotation)]
            axes_value[i].plot(d.date_key, d.stock_value, label="DM" if dm else "Non-DM")
            axes_value[i].legend()
            
        d = sla_value[(sla_value.rotation == rotation)]
        axes_value[i].plot(d.date_key, d.stock_value, label="All")
        axes_value[i].legend()
    fig_value.autofmt_xdate(); fig_value.tight_layout()

    
    df1.to_excel(stock_dc_writer, sheet_name=title, index=False)

In [ ]:
plot_stock_level_dc(stock_level_dc, 'all items')

In [ ]:
plot_stock_level_dc(stock_level_dc[stock_level_dc['holding_code'] == '002'], 'Nestle')

In [ ]:
plot_stock_level_dc(stock_level_dc[stock_level_dc['holding_code'] == '693'], 'P&G')

In [ ]:
plot_stock_level_dc(stock_level_dc[stock_level_dc['holding_code'] == '700'], 'Unilever')

In [ ]:
stock_dc_writer.save()
print(f'Please check file {stock_level_dc_file} for detail')

---

### 2.2  Stock level - store

In [ ]:
stock_level_store = spark.sql(get_query(
    'sql/kpi_stock_level_store.sql',
    # database_name='vartefact', date_start='20190630', date_end='20190730',
    database_name='vartefact', run_date=STOCK_LEVEL_END
)).toPandas()

In [ ]:
stock_store_writer = ExcelWriter(record_folder + stock_level_store_file)

In [ ]:
def plot_stock_level_store(df, title):
    start_day = pd.to_datetime('20190909', format='%Y%m%d')
    df1 = df.sort_values(by=['store_code', 'rotation', 'date_key']).copy()
    
    sl_all_stores = df1.groupby(['in_dm', 'rotation', 'date_key'])['stock_level'].sum().reset_index()
    sl_all_stores['date_key'] = pd.to_datetime(sl_all_stores.date_key, format='%Y%m%d')
    
    sla_all_stores = df1.groupby(['rotation', 'date_key'])['stock_level'].sum().reset_index()
    sla_all_stores['date_key'] = pd.to_datetime(sla_all_stores.date_key, format='%Y%m%d')
    
    fig, axes = plt.subplots(figsize=(14, 3.5), ncols=3)
    for i, rotation in enumerate(sl_all_stores.rotation.unique()):
        axes[i].set_title(f'Store stock for {title} Rotation {rotation}')
        axes[i].yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
        axes[i].axvline(start_day, ls="--")
        for dm in sl_all_stores.in_dm.unique():
            d = sl_all_stores[(sl_all_stores.in_dm == dm) & (sl_all_stores.rotation == rotation)]
            axes[i].plot(d.date_key, d.stock_level, label="DM" if dm else "Non-DM")
            axes[i].legend()
            
        d = sla_all_stores[sla_all_stores.rotation == rotation]
        axes[i].plot(d.date_key, d.stock_level, label="All")
        
        axes[i].legend()
    fig.autofmt_xdate(); fig.tight_layout()
    
    sl_value = df1.groupby(['in_dm', 'rotation', 'date_key'])['stock_value'].sum().reset_index()
    sl_value['date_key'] = pd.to_datetime(sl_value.date_key, format='%Y%m%d')
    
    sla_value = df1.groupby(['rotation', 'date_key'])['stock_value'].sum().reset_index()
    sla_value['date_key'] = pd.to_datetime(sla_value.date_key, format='%Y%m%d')
    
    fig_value, axes_value = plt.subplots(figsize=(14, 3.5), ncols=3)
    for i, rotation in enumerate(sl_all_stores.rotation.unique()):
        axes_value[i].set_title(f'Store stock value {title} Rotation {rotation}')
        axes_value[i].yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
        axes_value[i].axvline(start_day, ls="--")
        for dm in sl_value.in_dm.unique():
            d = sl_value[(sl_value.in_dm == dm) & (sl_value.rotation == rotation)]
            axes_value[i].plot(d.date_key, d.stock_value, label="DM" if dm else "Non-DM")
            axes_value[i].legend()
            
        d = sla_value[sla_value.rotation == rotation]
        axes_value[i].plot(d.date_key, d.stock_value, label="All")
        axes_value[i].legend()
            
    fig_value.autofmt_xdate(); fig_value.tight_layout()
    
    df1.to_excel(stock_store_writer, sheet_name=title, index=False)

In [ ]:
plot_stock_level_store(stock_level_store, "all items")

In [ ]:
plot_stock_level_store(stock_level_store[stock_level_store['con_holding'] == '002'], 'Nestle')

In [ ]:
plot_stock_level_store(stock_level_store[stock_level_store['con_holding'] == '693'], 'P&G')

In [ ]:
plot_stock_level_store(stock_level_store[stock_level_store['con_holding'] == '700'], 'Unilever')

In [ ]:
stock_store_writer.save()
print(f'Please check file {stock_level_store_file} for detail')

---

## 3 Service level

### 3.1 Service level - DC

In [ ]:
sl_writer = ExcelWriter(record_folder + service_level_file)

In [ ]:
sl_dc = read_query_and_fetch(
    'sql/kpi_service_level.sql',
    database_name='vartefact', date_start=SERVICE_LEVEL_START, date_end=SERVICE_LEVEL_END,
    kudu_replace={'lfms.daily_dctrxn': 'lfms_daily_dctrxn', 'lfms.ord': 'lfms_ord'}
)

In [ ]:
def show_service_level_dc(df):
    df1 = df.copy()
    for col in ['trxn_qty_sum', 'basic_order_qty_sum', 'service_level']:
        df1[col] = df1[col].astype('f8')

    df1 = df1.rename(columns={'trxn_qty_sum': 'received_qty_sum', 
                              'basic_order_qty_sum': 'ordered_qty_sum'})
    
    df1.to_excel(sl_writer, sheet_name='DC service level', index=False)
    return df1.groupby('holding_code')[['service_level']].mean().reset_index().style.hide_index()

In [ ]:
def plot_service_level_dc(df):
    start_day = pd.to_datetime('20190909', format='%Y%m%d')
        
    df1 = df.sort_values(by=['holding_code', 'date_key']).copy()
    df1['service_level'] = df1['service_level'].astype('f8')
    df1['date_key'] = pd.to_datetime(df1.date_key, format='%Y%m%d')
    
    df1 = df1.groupby(['holding_code', 'date_key'])['service_level'].mean().reset_index()
    
    fig, ax = plt.subplots(figsize=(12, 3))
    sp_002 = df1[df1.holding_code == '002']
    sp_693 = df1[df1.holding_code == '693']
    sp_700 = df1[df1.holding_code == '700']
    ax.plot(sp_002.date_key, sp_002.service_level, label='Nestle')
    ax.plot(sp_693.date_key, sp_693.service_level, label='P&G')
    ax.plot(sp_700.date_key, sp_700.service_level, label='Unilever')
    ax.axvline(start_day, ls="--")
    
    ax.legend()
    ax.set_title(f'DC service level')
    fig.autofmt_xdate()

In [ ]:
plot_service_level_dc(sl_dc)

In [ ]:
print(f'Average DC service level from {SERVICE_LEVEL_START} to {SERVICE_LEVEL_END}:')
show_service_level_dc(sl_dc)

### 3.2 Service level - Store

In [ ]:
sl_store = read_query_and_fetch(
    'sql/kpi_service_level_store.sql',
    database_name='vartefact', date_start=SERVICE_LEVEL_START, date_end=SERVICE_LEVEL_END,
   kudu_replace={'lfms.daily_shipment':'lfms_daily_shipment'}
)

In [ ]:
def show_service_level_store(df):
    df1 = df.copy()
    for col in ['order_qty_in_sku_sum', 'delivery_qty_in_sku_sum', 'service_level']:
        df1[col] = df1[col].astype('f8')

    df1 = df1.rename(columns={'delivery_qty_in_sku_sum': 'store_received_qty_sum', 
                              'order_qty_in_sku_sum': 'store_ordered_qty_sum'})
    
    df1.to_excel(sl_writer, sheet_name='Store service level', index=False)
    return df1.groupby(['holding_code','rotation', 'piece_picking'])[['service_level']] \
            .mean().reset_index().style.hide_index()


In [ ]:
def plot_service_level_store(df):
    start_day = pd.to_datetime('20190909', format='%Y%m%d')
    
    df1 = df.sort_values(by=['rotation', 'piece_picking', 'order_date']).copy()
    df1['service_level'] = df1['service_level'].astype('f8')
    
    sl = df1.groupby(['rotation', 'piece_picking', 'order_date'])['service_level'].mean().reset_index()
    sl['order_date'] = pd.to_datetime(sl.order_date, format='%Y%m%d')
    fig, axes = plt.subplots(figsize=(12, 4), ncols=3)
    for i, rotation in enumerate(['A', 'B', 'X']):
        axes[i].set_title(f'Store service level Rotation {rotation}')
        axes[i].axvline(start_day, ls="--")
        for psp in sl.piece_picking.unique():
            d = sl[(sl.piece_picking == psp) & (sl.rotation == rotation)]
            axes[i].plot(d.order_date, d.service_level, label="piece picking" if psp == "Y" else "regular")
            axes[i].legend()
    fig.autofmt_xdate(); fig.tight_layout()

In [ ]:
plot_service_level_store(sl_store)

In [ ]:
print(f'Average store service level from {SERVICE_LEVEL_START} to {SERVICE_LEVEL_END}:')
show_service_level_store(sl_store)

In [ ]:
sl_writer.save()

## 4 Out-of-Stock item list

In [ ]:
oos_writer = ExcelWriter(record_folder + oos_file)

In [ ]:
oos_item_list_dc = read_query_and_fetch(
    'sql/kpi_oos_item_list_dc.sql',
    # database_name='vartefact', date_start='20190701', date_end='20190730',
    database_name='vartefact', oos_check_date=OOS_CHECK_DATE,
    kudu_replace={'lfms.daily_dcstock': 'lfms_daily_dcstock'}
)

In [ ]:
print(f'{len(oos_item_list_dc)} out of stock DC items found')
oos_item_list_dc.to_excel(oos_writer, sheet_name='Out of stock items in DC', index=False)

In [ ]:
oos_item_list_store = read_query_and_fetch(
    'sql/kpi_oos_item_list_store.sql',
    # database_name='vartefact', date_start='20190701', date_end='20190730',
    database_name='vartefact', oos_check_date=OOS_CHECK_DATE,
    # kudu_replace={'lfms.daily_dcstock': 'lfms_daily_dcstock'}
)

In [ ]:
oos_item_store = oos_item_list_store.groupby(['store_code'])['full_item_code'].count() \
        .reset_index() \
        .sort_values(by=['full_item_code'], ascending = False) \
        .copy()

oos_item_store.columns = ["Store code", "Number of items out of stock"]

oos_item_store.to_excel(oos_writer, sheet_name='Out of stock in store', index=False)

In [ ]:
print("Top 10 stores with most out of stock items")
oos_item_store.head(10)

In [ ]:
oos_item = oos_item_list_store.groupby(['full_item_code', 'item_id', 'sub_id',
                                        'cn_name', 'rotation', 'ds_supplier_code',
                                       'store_status','dc_status',
                                       'item_stop_start_date', 'item_stop_end_date'])['store_code'].count() \
        .reset_index() \
        .sort_values(by=['store_code'], ascending = False) \
        .copy() \

oos_item.columns = ["Full item code", "Item ID", "Sub ID", 
                    "CN name", 'Rotation', 'DS Supplier Code',
                    "Store status","DC status", 
                    "Stop start date","Stop end date",
                    "Number out of stock stores"]

oos_item.to_excel(oos_writer, sheet_name='Out of stock items', index=False)

In [ ]:
print("Top 10 out of stock items")
oos_item.head(10)

In [ ]:
print(f'{len(oos_item_list_store)} out of stock store items found')
oos_item_list_store.to_excel(oos_writer, sheet_name='Out of stock items in store', index=False)

In [ ]:
no_predict_item_store = oos_item_list_store[oos_item_list_store["sales_prediction"] < 0]

In [ ]:
no_predict_item = no_predict_item_store[['full_item_code', 'item_id', 'sub_id', 
                                         'cn_name', 'rotation', 'con_holding', 'ds_supplier_code',
                                        'store_status','dc_status',
                                        'item_stop_start_date', 'item_stop_end_date']] \
                    .drop_duplicates()

In [ ]:
print("Items that does not have sales prediction")
no_predict_item.to_excel(oos_writer, sheet_name='No sales prediction item', index=False)
no_predict_item

In [ ]:
oos_writer.save()
print(f'Please check file {oos_file} for detail')

In [ ]:
T1 = time.time()

---

In [ ]:
print(f'Generating monitoring report takes {T1-T0:.2f} seconds.')